## Subgraph Classification with Multi-head attenttion - Variable Misuse

The pooling function is made of MULTIHEADATTENTION layers and Max pooling

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

[16:59:50] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: libtorch_cuda_cpp.so: cannot open shared object file: No such file or directory
Using backend: pytorch


In [3]:
# !pip install dgl-cu111==0.7.1 -f https://data.dgl.ai/wheels/repo.html

# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [4]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "10_percent_v1"
subgraph_partition = join(data_path, "partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

# Data distribution

In [5]:
labels = unpersist(filecontent_path)
for part in labels['partition'].unique():
    print(part)
    print(labels.query('partition == @part')['label'].value_counts())
    print()

train
Correct            35036
Variable misuse    35036
Name: label, dtype: int64

dev
Variable misuse    3754
Correct            3754
Name: label, dtype: int64

eval
Variable misuse    18787
Correct            18787
Name: label, dtype: int64



# Config

In [6]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    #use_edge_types=True,
    gpu=0,
    epochs=10, 
    
    train_frac=0.8,
    random_seed=42, 
    
    # model parameters
    elem_emb_size=300,
    node_emb_size=300,                  # *** dimensionality of node embeddings
    h_dim=300,                           # *** should match to node dimensionality
    n_layers=5,
    dropout=0.1,
    activation="relu"
)

In [7]:
save_config(config, "attention_pool_var_misuse_subgraph.yaml")

# Create Dataset

In [8]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

# Declare target loading function (labels)

In [9]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

One or several objectives could be used

In [10]:
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective, SubgraphClassifierObjectiveWithAttentionPooling

class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="VariableMisuseClfWithAtt_nb",
                objective_class=SubgraphClassifierObjectiveWithAttentionPooling,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

## Launch Tensorboard

In [11]:
# %tensorboard --logdir './2_percent_v1'

## Start model training and Evaluation

In [ ]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Number of nodes 4900346


2022-09-12 17:33:28.413430: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Epoch 1:   0%|                                          | 0/548 [00:00<?, ?it/s]

Epoch: 0, Time: 3532 s
{'Accuracy/test/VariableMisuseClfWithAtt_nb_': 0.5625037961613217,
 'Accuracy/train/VariableMisuseClfWithAtt_nb_': 0.5892857142857143,
 'Accuracy/train_avg/VariableMisuseClfWithAtt_nb': 0.5300870861574557,
 'Accuracy/val/VariableMisuseClfWithAtt_nb_': 0.5634395177562551,
 'Loss/test/VariableMisuseClfWithAtt_nb_': 0.6803851062748708,
 'Loss/train/VariableMisuseClfWithAtt_nb_': 0.6771707534790039,
 'Loss/train_avg/VariableMisuseClfWithAtt_nb': 0.6900643697620308,
 'Loss/val/VariableMisuseClfWithAtt_nb_': 0.6796820163726807,
 'grad_norm/train/_': 0.0807687588586647}


Epoch 2:   0%|                                          | 0/548 [00:00<?, ?it/s]

Epoch: 1, Time: 3517 s
{'Accuracy/test/VariableMisuseClfWithAtt_nb_': 0.5,
 'Accuracy/train/VariableMisuseClfWithAtt_nb_': 0.5,
 'Accuracy/train_avg/VariableMisuseClfWithAtt_nb': 0.5409015739051095,
 'Accuracy/val/VariableMisuseClfWithAtt_nb_': 0.5,
 'Loss/test/VariableMisuseClfWithAtt_nb_': 0.6931448568697689,
 'Loss/train/VariableMisuseClfWithAtt_nb_': 0.693170964717865,
 'Loss/train_avg/VariableMisuseClfWithAtt_nb': 0.6873976933042498,
 'Loss/val/VariableMisuseClfWithAtt_nb_': 0.6931439779572568,
 'grad_norm/train/_': 0.008150359573924433}


...

Epoch: 7, Time: 4425 s
{'Accuracy/test/VariableMisuseClfWithAtt_nb_': 0.746096027696793,
 'Accuracy/train/VariableMisuseClfWithAtt_nb_': 0.75,
 'Accuracy/train_avg/VariableMisuseClfWithAtt_nb': 0.7395785812043796,
 'Accuracy/val/VariableMisuseClfWithAtt_nb_': 0.7512674031476997,
 'Loss/test/VariableMisuseClfWithAtt_nb_': 0.48807985594077985,
 'Loss/train/VariableMisuseClfWithAtt_nb_': 0.5581600069999695,
 'Loss/train_avg/VariableMisuseClfWithAtt_nb': 0.4904422107404166,
 'Loss/val/VariableMisuseClfWithAtt_nb_': 0.4902320372856269,
 'grad_norm/train/_': 0.8128292223058166}


Epoch 9:   0%|                                          | 0/548 [00:00<?, ?it/s]

Epoch: 8, Time: 4117 s
{'Accuracy/test/VariableMisuseClfWithAtt_nb_': 0.7549494351311953,
 'Accuracy/train/VariableMisuseClfWithAtt_nb_': 0.7142857142857143,
 'Accuracy/train_avg/VariableMisuseClfWithAtt_nb': 0.755480562434828,
 'Accuracy/val/VariableMisuseClfWithAtt_nb_': 0.7555614406779662,
 'Loss/test/VariableMisuseClfWithAtt_nb_': 0.487361839535285,
 'Loss/train/VariableMisuseClfWithAtt_nb_': 0.5060354471206665,
 'Loss/train_avg/VariableMisuseClfWithAtt_nb': 0.4655713038722964,
 'Loss/val/VariableMisuseClfWithAtt_nb_': 0.49545697691076895,
 'grad_norm/train/_': 0.6213909322038}


Epoch 9: 100%|████████████████████████████████| 548/548 [47:16<00:00,  5.18s/it]


Epoch: 9, Time: 4120 s
{'Accuracy/test/VariableMisuseClfWithAtt_nb_': 0.7539442116132168,
 'Accuracy/train/VariableMisuseClfWithAtt_nb_': 0.7321428571428571,
 'Accuracy/train_avg/VariableMisuseClfWithAtt_nb': 0.7681443072210635,
 'Accuracy/val/VariableMisuseClfWithAtt_nb_': 0.7520555891848264,
 'Loss/test/VariableMisuseClfWithAtt_nb_': 0.5051959086032141,
 'Loss/train/VariableMisuseClfWithAtt_nb_': 0.49475106596946716,
 'Loss/train_avg/VariableMisuseClfWithAtt_nb': 0.44719680589045924,
 'Loss/val/VariableMisuseClfWithAtt_nb_': 0.5124638792821916,
 'grad_norm/train/_': 1.2542979829262497}


In [14]:
!nvidia-smi

Tue Sep 13 10:35:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   38C    P0    60W / 300W |  19934MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
config.save

AttributeError: 'dict' object has no attribute 'save'